In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv
import time

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

begin_time=time.time()

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 119ms
In Colab: False


In [2]:
from typing import Optional, Union, List, Dict, Any
from pydantic import BaseModel, ConfigDict, Field
from pandas import DataFrame

class Text2sqlQuestion(BaseModel): 
    model_config = ConfigDict(arbitrary_types_allowed=True)
    question: Optional[str] =None
    db_id: Optional[str] =None
    query: Optional[str] =None
    reasoning_type: Optional[str] =None
    commonsense_knowledge: Optional[str] =None
    schema: Optional[str] = None
    generated_query: Optional[str] =Field(None, description="The query generated by AI")
    system_output_df: Optional[str] = None
    gt_output_df: Optional[str] = None
    

/Users/gliozzo/Code/agentics911/agentics/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "Text2sqlQuestion" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [3]:
import re

def fix_double_quoted_literals(sql: str) -> str:
    """
    Convert double-quoted *literals* to single-quoted strings.
    Keep double-quoted *identifiers* like "MyTable" as-is.

    Heuristic: if the content is a simple identifier ([A-Za-z_][A-Za-z0-9_]*),
    we keep the double quotes; otherwise we treat it as a literal and convert.
    """
    ident_re = re.compile(r'^[A-Za-z_][A-Za-z0-9_]*$')

    def repl(m):
        body = m.group(1).replace('""', '"')  # unescape doubled quotes inside ""
        if ident_re.fullmatch(body):
            # looks like an identifier → leave as "Identifier"
            return f'"{m.group(1)}"'
        # looks like a literal → convert to '...'
        return "'" + body.replace("'", "''") + "'"

    # Match " ... " allowing doubled "" inside
    return re.sub(r'"((?:[^"]|"")*)"', repl, sql)

In [4]:
from pandas import DataFrame
import aiosqlite
import asyncio

async def async_execute_sql(sql_query: str, db_path:str) -> str:
        try:
            async with aiosqlite.connect(db_path) as db:
                async with db.execute(sql_query.replace("\"","'")) as cursor:
                    columns = [description[0] for description in cursor.description]
                    rows = await asyncio.wait_for(cursor.fetchall(), timeout=10)
                    df = DataFrame(rows, columns=columns)
                    return df.to_json()
        except Exception as e:
            return f"Error: {str(e)}"

In [5]:
import sqlite3
def get_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema_json={}
    for table in tables:

        cursor.execute(f"PRAGMA table_info({table[0]});")
        schema = cursor.fetchall()
        
        schema_json[table[0]] = { col[1] : {"type" : col[2], "notnull" : col[3], "dflt_value": col[4], } for col in schema} 
    return schema_json

res = get_schema("/Users/gliozzo/Data/Text2SQL/Archer/database/bike_1/bike_1.sqlite")
print(res)

{'station': {'id': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'name': {'type': 'TEXT', 'notnull': 0, 'dflt_value': None}, 'lat': {'type': 'NUMERIC', 'notnull': 0, 'dflt_value': None}, 'long': {'type': 'NUMERIC', 'notnull': 0, 'dflt_value': None}, 'dock_count': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'city': {'type': 'TEXT', 'notnull': 0, 'dflt_value': None}, 'installation_date': {'type': 'TEXT', 'notnull': 0, 'dflt_value': None}}, 'status': {'station_id': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'bikes_available': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'docks_available': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'time': {'type': 'TEXT', 'notnull': 0, 'dflt_value': None}}, 'trip': {'id': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'duration': {'type': 'INTEGER', 'notnull': 0, 'dflt_value': None}, 'start_date': {'type': 'TEXT', 'notnull': 0, 'dflt_value': None}, 'start_station_name': {'type': 'TEXT', 'notnul

In [6]:

from typing import Set
import pandas as pd


def remove_duplicate_col_df(df):
    return df.loc[:, ~df.columns.duplicated()]


def convert_df_to_set(df, row_invariant=True) -> Set:
    # remove duplicate columns
    df = remove_duplicate_col_df(df)

    if row_invariant:
        return set(
            [
                tuple(sorted(df[c].to_list(), key=lambda x: (x is None, str(x))))
                for c in df.columns.values
            ]
        )
    else:
        return set([tuple(df[c].to_list()) for c in df.columns.values])



def compare_df(gt, predicted, row_invariant=False) -> bool:
    # 1: gt_df is subset of predicted_df
    # 2: df1 == df2
    # 0: otherwise
    if predicted.startswith("Error:") or gt.startswith("Error:"): return 0
    gt_df = pd.read_json(gt)
    predicted_df = pd.read_json(predicted)
    gt_df = gt_df.map(lambda x: float(f"{x:.5f}") if isinstance(x, float) else x)
    predicted_df = predicted_df.map(
        lambda x: float(f"{x:.5f}") if isinstance(x, float) else x
    )

    gt_set = convert_df_to_set(gt_df, row_invariant=row_invariant)
    predicted_set = convert_df_to_set(predicted_df, row_invariant=row_invariant)

    intersec = gt_set & predicted_set
    return (
        1
        if (intersec == gt_set)
        else 1
        if (predicted_set == gt_set)
        else 1
        if (intersec == predicted_set)
        else 0
    )


In [7]:
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("execute_sql_query")
async def execute_sql_query(sql_query:str, db_id:str)-> str:
    """Execute a SQL query against the target db and return the execution results (error or json dataframe)"""
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            db_id,db_id+".sqlite" )
    system_output_df= await async_execute_sql(sql_query, schema_path)
    return system_output_df 


In [8]:
from agentics import Agentics as AG
from agentics.core.llm_connections import get_llm_provider

training = AG.from_jsonl("/Users/gliozzo/Data/Text2SQL/Archer/en_data/dev.json", 
                        jsonl=False, 
                        atype=Text2sqlQuestion,
                        #max_rows=1,
                        )
training.llm=get_llm_provider("watsonx")
training.reasoning=False
training.tools=[execute_sql_query]
training.max_iter=10
training[0].model_dump_json()

2025-09-13 12:03:08.318 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-13 12:03:08.318 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-13 12:03:08.318 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-13 12:03:08.319 | DEBUG    | agentics.core.llm_connections:<module>:133 - 2 - OpenAI
2025-09-13 12:03:08.319 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-13 12:03:08.325 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-13 12:03:08.326 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-13 12:03:08.328 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider

'{"question":"List the name and age of each singer at the time of the release of the song \\"Gentleman\\".","db_id":"concert_singer","query":"SELECT Name , Age + ( SELECT Song_release_year FROM singer WHERE Song_Name = \\"Gentleman\\" ) - strftime ( \\"%Y\\" , \\"now\\" ) AS target_age FROM singer","reasoning_type":"- + C","commonsense_knowledge":"Age is calculated by substracting date of birth from a given date.","schema":null,"generated_query":null,"system_output_df":null,"gt_output_df":null}'

In [9]:
async def get_schema_map(state:Text2sqlQuestion)-> Text2sqlQuestion:
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            state.db_id,state.db_id+".sqlite" )
    state.schema=str(get_schema(schema_path))
    print(state.db_id)
    return state

training= await training.amap(get_schema_map)
training.verbose_agent=False
print(training[0].schema)

2025-09-13 12:03:08.341 | DEBUG    | agentics.core.agentics:amap:207 - Executing amap on function <function get_schema_map at 0x169fb8b80>
2025-09-13 12:03:08.346 | DEBUG    | agentics.core.agentics:amap:232 - 20 states processed. 0.00021255016326904297 seconds average per state in the last chunk ...
2025-09-13 12:03:08.349 | DEBUG    | agentics.core.agentics:amap:232 - 40 states processed. 0.00013870000839233398 seconds average per state in the last chunk ...
2025-09-13 12:03:08.353 | DEBUG    | agentics.core.agentics:amap:232 - 60 states processed. 0.00020110607147216797 seconds average per state in the last chunk ...
2025-09-13 12:03:08.358 | DEBUG    | agentics.core.agentics:amap:232 - 80 states processed. 0.0002407550811767578 seconds average per state in the last chunk ...
2025-09-13 12:03:08.362 | DEBUG    | agentics.core.agentics:amap:232 - 100 states processed. 0.00015285015106201172 seconds average per state in the last chunk ...
2025-09-13 12:03:08.363 | DEBUG    | agentics.

concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
concert_singer
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
world_1
worl

In [10]:
training = await training.self_transduction(["question","schema","commonsense_knowledge", "db_id"], 
                            ["generated_query"], 
                            instructions="Generate a SQL query from the input question and target db schema")
print(training[0].generated_query)

2025-09-13 12:03:08.372 | DEBUG    | agentics.core.agentics:__lshift__:519 - Executing task: Generate a SQL query from the input question and target db schema
104 states will be transduced
2025-09-13 12:03:08.373 | DEBUG    | agentics.core.agentics:__lshift__:613 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-13 12:03:54.562 | DEBUG    | agentics.core.agentics:__lshift__:650 - Processed 20 states in 46.188432693481445 seconds
2025-09-13 12:03:54.563 | DEBUG    | agentics.core.agentics:__lshift__:702 - 20 states processed in 2.3094216346740724 seconds average per state ...
2025-09-13 12:04:11.390 | DEBUG    | agentics.core.agentics:__lshift__:650 - Processed 20 states in 16.82680082321167 seconds
2025-09-13 12:04:11.391 | DEBUG    | agentics.core.agentics:__lshift__:702 - 40 states processed in 0.8413400411605835 seconds average per state ...
2025-09-13 12:04:27.854 | DEBUG    | agentics.core.agentics:__lshift__:650 - Processed 2

SELECT Name, Age FROM singer WHERE Song_Name = "Gentleman"


In [11]:
async def execute_query_map(state:Text2sqlQuestion)-> Text2sqlQuestion:
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            state.db_id,state.db_id+".sqlite" )
    state.system_output_df= await async_execute_sql(state.generated_query, schema_path)
    state.gt_output_df= await async_execute_sql(state.query, schema_path)
    return state

training= await training.amap(execute_query_map)
print(training[0].gt_output_df)


2025-09-13 12:05:11.975 | DEBUG    | agentics.core.agentics:amap:207 - Executing amap on function <function execute_query_map at 0x16db0e480>
2025-09-13 12:05:12.001 | DEBUG    | agentics.core.agentics:amap:232 - 20 states processed. 0.0012450575828552245 seconds average per state in the last chunk ...
2025-09-13 12:05:12.018 | DEBUG    | agentics.core.agentics:amap:232 - 40 states processed. 0.0008552432060241699 seconds average per state in the last chunk ...
2025-09-13 12:05:12.034 | DEBUG    | agentics.core.agentics:amap:232 - 60 states processed. 0.0007887601852416993 seconds average per state in the last chunk ...
2025-09-13 12:05:12.073 | DEBUG    | agentics.core.agentics:amap:232 - 80 states processed. 0.0019455075263977051 seconds average per state in the last chunk ...
2025-09-13 12:05:12.095 | DEBUG    | agentics.core.agentics:amap:232 - 100 states processed. 0.0010588526725769042 seconds average per state in the last chunk ...
2025-09-13 12:05:12.099 | DEBUG    | agentics.c

{"Name":{"0":"Joe Sharp","1":"Timbaland","2":"Justin Brown","3":"Rose White","4":"John Nizinik","5":"Tribal King"},"target_age":{"0":41,"1":21,"2":18,"3":30,"4":32,"5":14}}


In [12]:
training.to_jsonl("/tmp/anker_task.jsonl")

2025-09-13 12:05:12.106 | DEBUG    | agentics.core.agentics:to_jsonl:940 - Exporting 104 Agentics to CSV /tmp/anker_task.jsonl


In [13]:
print(f"task executed in {time.time() - begin_time} seconds")

task executed in 126.62181782722473 seconds


In [14]:
training = AG.from_jsonl("/tmp/anker_task.jsonl")
total = 0
for question in training:
    total+= compare_df(question.system_output_df, question.gt_output_df)
    print(question.gt_output_df, question.system_output_df, question.generated_query,compare_df(question.system_output_df, question.gt_output_df) )
print(f"Test size: {len(training.states)}\nExecution Accuracy: {total/len(training.states)}")

{"Name":{"0":"Joe Sharp","1":"Timbaland","2":"Justin Brown","3":"Rose White","4":"John Nizinik","5":"Tribal King"},"target_age":{"0":41,"1":21,"2":18,"3":30,"4":32,"5":14}} {"Name":{"0":"John Nizinik"},"Age":{"0":43}} SELECT Name, Age FROM singer WHERE Song_Name = "Gentleman" 0
{"Name":{"0":"Joe Sharp","1":"Timbaland","2":"Justin Brown","3":"Rose White","4":"John Nizinik","5":"Tribal King"},"target_age":{"0":41,"1":21,"2":18,"3":30,"4":32,"5":14}} {"Name":{"0":"John Nizinik","1":"John Nizinik"},"Song_release_year":{"0":"2014","1":"2014"},"Age":{"0":43,"1":43}} SELECT s.Name, s.Song_release_year, s.Age FROM singer s INNER JOIN singer_in_concert sic ON s.Singer_ID = sic.Singer_ID WHERE s.Song_Name = 'Gentleman' 0
{"Name":{"0":"Joe Sharp","1":"Timbaland","2":"Justin Brown","3":"Rose White","4":"John Nizinik","5":"Tribal King"},"target_age":{"0":28,"1":8,"2":5,"3":17,"4":19,"5":1}} {"Name":{},"Age":{}} SELECT s.Name, 2001 - s.Age AS Age FROM singer s WHERE s.Song_Name = 'Gentleman' AND s.S

In [42]:
training = AG.from_jsonl("/Users/gliozzo/Data/Text2SQL/Archer/en_data/train.json",jsonl=False)
reasoning_types={}
import random
for question in training:
    for reasoning_type in question.reasoning_type.split(" "):
        if not reasoning_type in reasoning_types: 
            reasoning_types[reasoning_type] = []
        reasoning_types[reasoning_type].append(question)
for reasoning_type in reasoning_types:
    print(random.choice(reasoning_types[reasoning_type]))

db_id='bike_1' query='SELECT end_station_name , MAX ( duration ) AS max_duration , MIN ( duration ) AS min_duration , MAX ( duration ) - MIN ( duration ) AS diff FROM trip GROUP BY end_station_name' question='List the names of all stations, and calculate the minimum duration, maximum duration and difference between the two for trips ending at each respective station.' reasoning_type='-' commonsense_knowledge=''
db_id='riding_club' query='SELECT A.Player_name , A.Votes - 1000 AS diff FROM player A JOIN player_coach B ON A.Player_ID = B.Player_ID JOIN coach C ON B.Coach_ID = C.Coach_ID JOIN club D ON C.Club_ID = D.Club_ID WHERE D.Start_year < 2006' question='What are the names of players who are guided by coaches from clubs established before the 21st Century, and how much higher are their votes than the lowest if the lowest votes count is 1000?' reasoning_type='- C H' commonsense_knowledge='The 21st century begins with 1 January 2001.'
db_id='hospital_1' query='SELECT ( strftime ( "%s" 